In [2]:
import pandas as pd

In [1]:
# Only use 7 columns (don't use observation count)
# filter for us birds
# Rename columns
# Drop nulls
# Clean up badly named birds
# convert date column to datetime object
# extract month from date
# map month column to season
# combine county and state

## Clean up for regions
# rename columns
# Strip leading whitespace from State
# drop numbers from regionName
# drop state from countyname
# Map alaska county names
# Capitalize city in St. Louis and Richmond works
# drop parish and county from CountyName
# combine county and state

# merge dfs
# create pivot table of rarities
# county-region mapping dict
# state-county mapping dict
# list of birds, seasons, and states


In [3]:
df = pd.read_csv("~/Desktop/repos/Bird-Check/bird_data.csv", 
                 sep='\t', 
                 nrows=10000, 
                 usecols=['COMMON NAME', 'COUNTRY', 'STATE', 'COUNTY',
                          'LATITUDE', 'LONGITUDE'])

print(df.shape)

df.head()

(10000, 6)


,COMMON NAME,COUNTRY,STATE,COUNTY,LATITUDE,LONGITUDE
0,Magnolia Warbler,United States,Illinois,Cook,41.775629,-87.583273
1,White-rumped Sandpiper,Canada,Quebec,Manicouagan,49.216670,-68.150000
2,Common Scoter,Sweden,Hallands län [SE-13],NaN,57.065084,12.243579
3,Ring-billed Gull,Canada,Manitoba,South Interlake,50.193256,-97.137935
4,Red-winged Blackbird,Canada,Manitoba,South Interlake,50.193256,-97.137935


In [6]:
def us_bird_filter(df):
    return df.query("country == 'United States'")

def bird_column_renamer(df):
    return df.rename(columns={
    'COMMON NAME': 'name',
    'OBSERVATION COUNT': 'observ_count',
    'COUNTRY': 'country',
    'STATE': 'state',
    'COUNTY': 'county',
    'LATITUDE': 'latitude',
    'LONGITUDE': 'longitude'})


def bad_name_cleaner(df):
    pass

Index(['name', 'country', 'state', 'county', 'latitude', 'longitude'], dtype='object')

In [2]:
def clean_regions(df):
    
    # Copy df 
    df = df.copy()
    # rename columns
    df = region_column_renamer(df)
    
    # strip leading whitespace from state column
    # drop numbers from region
    # drop state from county
    # clean up state, county, and region columns appropriately
    df = clean_cols(df)
    
   
    # fix county names
    df = county_fixer(df)
    
    # combine county state
    df = county_state_merger(df)
    
    return df

In [3]:
def clean_chunks(df):
    
    # rename columns
    df = region_column_renamer(df)
        
    # filter for us birds
    df = us_bird_filter(df)

    df.dropna(subset=['county'])
    
    # clean badly named birds
    df = bad_name_cleaner(df)
    
    # get season thru date
    # convert date to datetime
    # extract month
    # map to a season
    df = get_season(df)

    # combine county + state
    df = county_state_merger(df)